In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch


base_model = AutoModelForCausalLM.from_pretrained(
    'meta-llama/Llama-2-7b-chat-hf',
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.float16,   # optional if you have enough VRAM
)
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-chat-hf')

model = PeftModel.from_pretrained(base_model, 'FinGPT/fingpt-forecaster_dow30_llama2-7b_lora')
model = model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
prompt = """
[Company Introduction]:

{name} is a leading entity in the {finnhubIndustry} sector. Incorporated and publicly traded since {ipo}, the company has established its reputation as one of the key players in the market. As of today, {name} has a market capitalization of {marketCapitalization:.2f} in {currency}, with {shareOutstanding:.2f} shares outstanding. {name} operates primarily in the {country}, trading under the ticker {ticker} on the {exchange}. As a dominant force in the {finnhubIndustry} space, the company continues to innovate and drive progress within the industry.

From {startDate} to {endDate}, {name}'s stock price {increase/decrease} from {startPrice} to {endPrice}. Company news during this period are listed below:

[Headline]: ...
[Summary]: ...

[Headline]: ...
[Summary]: ...

Some recent basic financials of {name}, reported at {date}, are presented below:

[Basic Financials]:
{attr1}: {value1}
{attr2}: {value2}
...

Based on all the information before {curday}, let's first analyze the positive developments and potential concerns for {symbol}. Come up with 2-4 most important factors respectively and keep them concise. Most factors should be inferred from company-related news. Then make your prediction of the {symbol} stock price movement for next week ({period}). Provide a summary analysis to support your prediction.

"""

In [3]:
SYSTEM_PROMPT = "You are a seasoned stock market analyst. Your task is to list the positive developments and potential concerns for companies based on relevant news and basic financials from the past weeks, then provide an analysis and prediction for the companies' stock price movement for the upcoming week. Your answer format should be as follows:\n\n[Positive Developments]:\n1. ...\n\n[Potential Concerns]:\n1. ...\n\n[Prediction & Analysis]:\n...\n"

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

prompt = B_INST + B_SYS + SYSTEM_PROMPT + E_SYS + 'NVDA' + E_INST
inputs = tokenizer(
    prompt, return_tensors='pt'
)
inputs = {key: value.to(model.device) for key, value in inputs.items()}
        
res = model.generate(
    **inputs, max_length=4096, do_sample=True,
    eos_token_id=tokenizer.eos_token_id,
    use_cache=True
)
output = tokenizer.decode(res[0], skip_special_tokens=True)

In [4]:
inputs, output

({'input_ids': tensor([[    1,   518, 25580, 29962,  9314, 14816, 29903,  6778,    13,  3492,
             526,   263,  4259,   287, 10961,  9999,  3483,   858, 29889,  3575,
            3414,   338,   304,  1051,   278,  6374,  2693,  1860,   322,  7037,
           21838,   363, 14582,  2729,   373,  8018,  9763,   322,  6996, 22347,
            1338,   515,   278,  4940, 11405, 29892,   769,  3867,   385,  7418,
             322, 18988,   363,   278, 14582, 29915, 10961,  8666, 10298,   363,
             278,   701, 11506,  4723, 29889,  3575,  1234,  3402,   881,   367,
             408,  4477, 29901,    13,    13, 29961,  9135,  3321, 10682,  1860,
            5387,    13, 29896, 29889,  2023,    13,    13, 29961, 29925,   327,
            2556, 23924,   824, 29879,  5387,    13, 29896, 29889,  2023,    13,
              13, 29961, 23084,  2463,   669, 24352,  5387,    13,   856,    13,
              13, 29966,   829, 14816, 29903,  6778,    13,    13, 29940, 29963,
            769